In [7]:
import os
import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from sklearn.model_selection import StratifiedKFold

In [8]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [9]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [10]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [11]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [12]:
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

# Az adatok újraformázása
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

# Adatok szétosztása tanítási és validációs halmazokra
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, stratify=y_data, random_state=42)

# Modell létrehozása egyszer
model = model_builder.build_conv_pool_model(input_size, len(df_names))

# Early stopping beállítása
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Modell tanítása a teljes adathalmazon
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    callbacks=[early_stopping],
                    validation_data=(X_val, y_val),
                    verbose=1)

# Validációs eredmények kiírása
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

# Modell értékelése az összes adaton (tanítási + validációs halmazokon)
score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")

Epoch 1/20
1330/1330 [==============================] - 22s 17ms/step - loss: 1.0745 - accuracy: 0.7321 - val_loss: 0.5239 - val_accuracy: 0.8368
Epoch 2/20
1330/1330 [==============================] - 36s 27ms/step - loss: 0.5649 - accuracy: 0.8276 - val_loss: 0.4420 - val_accuracy: 0.8504
Epoch 3/20
1330/1330 [==============================] - 18s 13ms/step - loss: 0.4519 - accuracy: 0.8513 - val_loss: 0.3551 - val_accuracy: 0.8745
Epoch 4/20
1330/1330 [==============================] - 19s 14ms/step - loss: 0.3938 - accuracy: 0.8652 - val_loss: 0.3887 - val_accuracy: 0.8669
Epoch 5/20
1330/1330 [==============================] - 18s 14ms/step - loss: 0.3521 - accuracy: 0.8753 - val_loss: 0.3182 - val_accuracy: 0.8909
Epoch 6/20
1330/1330 [==============================] - 18s 14ms/step - loss: 0.3280 - accuracy: 0.8829 - val_loss: 0.3443 - val_accuracy: 0.8805
Epoch 7/20
1330/1330 [==============================] - 18s 14ms/step - loss: 0.3009 - accuracy: 0.8911 - val_loss: 0.3015 -

In [13]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [14]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 6ms/step


In [17]:
model.save("models/model_9049")

INFO:tensorflow:Assets written to: models/model_9049/assets


INFO:tensorflow:Assets written to: models/model_9049/assets


In [16]:
filtered_df = test_df[test_df['pred'] == '9']
filtered_df[['label', 'pred']]

,label,pred
23,Test3002.png,9
33,Test4779.png,9
82,Test5117.png,9
123,Test5739.png,9
145,Test3406.png,9
...,...,...
6876,Test3185.png,9
6886,Test3806.png,9
6956,Test4370.png,9
6973,Test5052.png,9
